In [1]:
import asyncio
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

from graph.builder import build_graph
from db_connection import setup_database, store, saver, close_pool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

async def main():
    try:
        # Setup database and get store/saver instances
        print("🔧 Setting up database...")
        db_store, db_saver = await setup_database()
        print("✅ Database setup complete!\n")
        print(f"Store: {db_store}")
        print(f"Saver: {db_saver}\n")

        # Build graph with checkpointer and store
        print("🏗️  Building graph...")
        graph = await build_graph()
        print("✅ Graph built!\n")

        # Configuration for this conversation thread
        config = {
            "configurable": {
                "thread_id": "test_thread_1",
                "user_id": "user_123"
            }
        }

        # Test message
        input_messages = [
            HumanMessage(content="My name is Lance. I live in SF with my wife. I have a 1 year old daughter.")
        ]

        print("🚀 Running graph and persisting memory to Postgres...\n")
        print("-" * 60)
        
        # Stream the graph execution
        async for chunk in graph.astream({"messages": input_messages}, config=config):
            if "messages" in chunk and chunk["messages"]:
                chunk["messages"][-1].pretty_print()
        
        print("-" * 60)
        print("\n✅ Run completed — memory persisted in Postgres!")
        
        # Optional: Test memory retrieval
        print("\n🔍 Testing memory retrieval...")
        state = await graph.aget_state(config)
        print(f"Messages in state: {len(state.values.get('messages', []))}")
        
        # Test another message to verify memory persistence
        print("\n🧪 Testing memory persistence with follow-up...")
        follow_up = [HumanMessage(content="What's my daughter's age?")]
        
        async for chunk in graph.astream({"messages": follow_up}, config=config):
            if "messages" in chunk and chunk["messages"]:
                chunk["messages"][-1].pretty_print()
        
        print("\n✅ Memory test complete!")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Clean up
        print("\n🧹 Closing database connection...")
        await close_pool()
        print("✅ Cleanup complete!")

if __name__ == "__main__":
    # Check if we're in a Jupyter notebook
    try:
        get_ipython()  # This will exist in Jupyter/IPython
        # Use the existing event loop
        await main()
    except NameError:
        # Not in Jupyter, use asyncio.run()
        asyncio.run(main())

c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔧 Setting up database...
✅ Database setup complete!

Store: <langgraph.store.postgres.aio.AsyncPostgresStore object at 0x000001819DE737F0>
Saver: <langgraph.checkpoint.postgres.aio.AsyncPostgresSaver object at 0x000001819DEC3E50>

🏗️  Building graph...


Vector extension setup: password authentication failed for user "postgres"
c:\Users\HP\Desktop\WAHA\src\rag\knowledge_base.py:148: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(
Failed to create vector store: (sqlalchemy.dialects.postgresql.asyncpg.ProgrammingError) <class 'asyncpg.exceptions.PostgresSyntaxError'>: cannot insert multiple commands into a prepared statement
[SQL: SELECT pg_advisory_xact_lock(1573678846307946496);CREATE EXTENSION IF NOT EXISTS vector;]
(Background on this error at: https://sqlalche.me/e/20/f405)
Traceback (most recent call last):
  File "c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\sqlalchemy\dialects\postgresql\a

✅ Graph built!

🚀 Running graph and persisting memory to Postgres...

------------------------------------------------------------

❌ Error: Store not found in config. Make sure graph is compiled with store parameter.

🧹 Closing database connection...
✅ Cleanup complete!


Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_9060\2706730387.py", line 42, in main
    async for chunk in graph.astream({"messages": input_messages}, config=config):
  File "c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\langgraph\pregel\main.py", line 3000, in astream
    async for _ in runner.atick(
  File "c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\langgraph\pregel\_runner.py", line 304, in atick
    await arun_with_retry(
  File "c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\langgraph\pregel\_retry.py", line 137, in arun_with_retry
    return await task.proc.ainvoke(task.input, config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\langgraph\_internal\_runnable.py", line 705, in ainvoke
    input = await asyncio.create_task(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HP\Desktop\WAHA\.venv\Lib\site-packages\langgraph\_internal\_runnable.py", line 473, in ai

In [ ]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)
await client.assistants.search()

In [ ]:
from langgraph.pregel.remote import RemoteGraph
from langchain_core.messages import convert_to_messages
from langchain_core.messages import HumanMessage, SystemMessage

# Get remote graph
url = "http://localhost:8123"
graph_name = "chatbot" 
remote_graph = RemoteGraph(graph_name, url=url)

In [ ]:
# Create a thread
thread = await client.threads.create()

In [ ]:
# Check any existing runs on a thread
thread = await client.threads.create()
runs = await client.runs.list(thread["thread_id"])
print(runs)

In [ ]:
# Ensure we've ask some questions and saved them to my user_id
user_input = "Hi, my name is Kanny."
config = {"configurable": {"user_id": "Kanny"}}
graph_name = "chatbot" 
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages": [HumanMessage(content=user_input)]}, config=config)

In [ ]:
thread = await client.threads.create()
user_input = "Tell me about your best dish."
config = {"configurable": {"user_id": "Kanny"}}
graph_name = "chatbot" 
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages": [HumanMessage(content=user_input)]}, config=config)

In [ ]:
print(run)

In [ ]:
# Check the run status
print(await client.runs.get(thread["thread_id"], run["run_id"]))

In [ ]:
user_input = "Hello, do you have pancakes?."
async for chunk in client.runs.stream(thread["thread_id"], 
                                      graph_name, 
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      config=config,
                                      stream_mode="messages-tuple"):

    if chunk.event == "messages":
        print("".join(data_item['content'] for data_item in chunk.data if 'content' in data_item), end="", flush=True)

In [ ]:
print(chunk)

In [ ]:
thread_state = await client.threads.get_state(thread['thread_id'])
for m in convert_to_messages(thread_state['values']['messages']):
    m.pretty_print()

In [ ]:
# Get the history of the thread
states = await client.threads.get_history(thread['thread_id'])

# Pick a state update to fork
to_fork = states[-2]
to_fork['values']

In [ ]:
items = await client.store.search_items(
    ("dish", "Kanny"),
    limit=5,
    offset=0
)
items['items']

In [ ]:
assistant = await client.assistants.create(
    "chatbot",
    config = {"configurable": {"user_id": "Kanny"}}
)
print(assistant)

In [ ]:
print(chunk)